### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.linear_model import LogisticRegression

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

baby_df = pd.read_csv('amazon_baby.csv')
baby_df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [3]:
#b)Replace all missing (nan) reviews with empty "" string
"""firstly we repalce all missing reviews with empty strings,
so we do not have any "garbage" data when removing punctuation"""

baby_df["review"] = baby_df["review"].fillna("")

# a)Remove punctuation from reviews using the given function
baby_df["review"].apply(remove_punctuation)

#short test: 
print(baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock')
print(remove_punctuation(baby_df["review"][4]) == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock')

False
True


In [4]:
#b)
"""
implemented above
"""

#short test:
baby_df["review"][38] == baby_df["review"][38]

True

In [5]:
#c) Drop all the entries with rating = 3, as they have neutral sentiment
baby_df = baby_df.drop(baby_df[baby_df["rating"] == 3].index)

#short test:
sum(baby_df["rating"] == 3)

0

In [6]:
#d) Set all positive >= 4 ratings to 1 and negative <= 2 to -1
"""
Here we map date to either positive or negative
This is why we had to drop "3". In this case it was neutral
"""

baby_df["rating"] = np.where(baby_df["rating"] < 3, -1, 1)
#short test:
sum(baby_df["rating"]**2 != 1) # no element is different from 1

0

## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names_out())
print(X_train_example.todense())



['adore' 'and' 'apples' 'bananas' 'dislike' 'hate' 'like' 'oranges' 'they'
 'we']
[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


In [8]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(X_test_example.todense())

[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2 
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer. 

In [9]:
#a)Split dataset into training and test sets.
train_df = baby_df.sample(frac=.8, random_state=1)
test_df = baby_df.drop(train_df.index)

In [10]:
#b)Transform reviews into vectors using CountVectorizer. 
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train_example = vectorizer.fit_transform(train_df.review)

# only transform on test

print(vectorizer.get_feature_names_out()[2000:3000])
# print(X_train_example.todense())
print(vectorizer.get_feature_names_out().shape)
print(X_train_example.shape) #too big to do todense()
"""
size it too big, so we cant do .todense
Instead I print small portion of dictionary
"""

['5month' '5months' '5mos' '5mth' '5mths' '5nor' '5ounce' '5oz' '5ozs'
 '5pk' '5pm' '5point' '5pros' '5pt' '5s' '5star' '5stars' '5t' '5th'
 '5ths' '5times' '5v' '5vdc' '5weeks' '5wk' '5wo' '5x' '5x11' '5x5' '5x6'
 '5x7' '5x7x2' '5x8' '5x9' '5y' '5year' '5years' '5yo' '5yr' '5yrs' '60'
 '600' '600d' '600ft' '600mm' '602' '60258' '607' '60db' '60ft' '60in'
 '60lbs' '60m' '60mins' '60ml' '60mph' '60s' '60th' '60x49x9' '60z' '61'
 '611' '61514' '61516' '615am' '6161948' '62' '622401a' '6237'
 '6237medwards' '625' '625mhz2412mhz2415' '625mhz2430mhz2433'
 '625mhz2448mhz2451' '625mhz2466mhz2469' '626' '62f' '62lb' '63' '630'
 '6346515description' '638' '64' '640x480' '646' '65' '650' '65039'
 '650and' '65279' '65281' '65289' '65292' '65306' '65311' '6552' '6586'
 '65cs' '65db' '65dlx' '65f' '65lb' '65lbs' '65pound' '66' '66lbs' '67'
 '68' '68f' '69' '692' '693' '699' '6a' '6am' '6c' '6f' '6ft' '6ft1'
 '6ft1in' '6hr' '6hrs' '6i' '6ib' '6in' '6inch' '6ish' '6kg' '6lb' '6lb12'
 '6lbs' '6lbs4' '

'\nsize it too big, so we cant do .todense\nInstead I print small portion of dictionary\n'

## Exercise 3 
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [12]:
#a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were)
"""
I had to change max_iter because model was failing do converge
"""
model = LogisticRegression(max_iter=1200)
model.fit(X_train_example, train_df["rating"])


LogisticRegression(max_iter=1200)

In [18]:
#b) Print 10 most positive and 10 most negative words
sorted_coef, sorted_features = (list(x) for x in zip(*sorted(zip(model.coef_.tolist()[0], vectorizer.get_feature_names_out()))))

top_positive = sorted_features[-10:]
top_negative = sorted_features[:10]

print(top_positive)
print(top_negative)
#hint: model.coef_, vectorizer.get_feature_names()

['wonderfully', 'minor', 'saves', 'vminnovations', 'thankful', 'amazing', 'lifesaver', 'amazed', 'pleasantly', 'rich']
['dissapointed', 'disappointing', 'worst', 'unusable', 'poorly', 'worthless', 'useless', 'returning', 'theory', 'disappointed']


## Exercise 4 
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [ ]:
#a)


In [ ]:
#b)

#hint: model.predict_proba()

In [ ]:
#c) 

#hint: use the results of b)

In [ ]:
#d) 


## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [ ]:
significant_words = ['love','great','easy','old','little','perfect','loves','well','able','car','broke','less','even','waste','disappointed','work','product','money','would','return']

In [ ]:
#a)


In [ ]:
#b)


In [ ]:
#c)

#hint: %time, %timeit